In [1]:
import numpy as np
import pandas as pd

In [3]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [5]:
df = pd.read_csv("covid_toy.csv")

In [7]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


gender, city -> nominal categories -one hot encoding\
cough -> ordinal encoding -ordinal encoding

In [11]:
df['cough'].value_counts()

cough
Mild      62
Strong    38
Name: count, dtype: int64

In [15]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [17]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
    df.drop(columns=['has_covid']),
    df['has_covid'],
    test_size=0.2
)

In [19]:
x_train

,age,gender,fever,cough,city
71,75,Female,104.0,Strong,Delhi
83,17,Female,104.0,Mild,Kolkata
9,64,Female,101.0,Mild,Delhi
21,73,Male,98.0,Mild,Bangalore
4,65,Female,101.0,Mild,Mumbai
...,...,...,...,...,...
63,10,Male,100.0,Mild,Bangalore
29,34,Female,NaN,Strong,Mumbai
6,14,Male,101.0,Strong,Bangalore
84,69,Female,98.0,Strong,Mumbai


# without column transformers

In [32]:
# adding simple imputer to fever col
# simple imputer will replace all null values with the mean of fever
si = SimpleImputer()
x_train_fever = si.fit_transform(x_train[['fever']])

x_test_fever = si.transform(x_test[['fever']])

x_train_fever.shape

(80, 1)

In [38]:
# ordinal encoding -> cough
oe = OrdinalEncoder(categories=[['Mild', 'Strong']])
x_train_cough = oe.fit_transform(x_train[['cough']])

x_test_cough = oe.transform(x_test[['cough']])
x_train_cough.shape

(80, 1)

In [54]:
# oneHot encoding -> gender, city
ohe = OneHotEncoder(drop='first', sparse=False)
x_train_gender_city = ohe.fit_transform(x_train[['gender', 'city']])
x_test_gender_city = ohe.transform(x_test[['gender', 'city']])
x_train_gender_city.shape

C:\Users\ADMIN\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(80, 4)

In [56]:
x_train_age = x_train.drop(columns=['gender', 'fever', 'cough', 'city']).values
x_test_age = x_test.drop(columns=['gender', 'fever', 'cough', 'city']).values

x_train_age.shape

(80, 1)

In [58]:
x_train_transformed = np.concatenate((x_train_age, x_train_fever, x_train_gender_city, x_train_cough ), axis=1)
x_test_transformed = np.concatenate((x_test_age, x_test_fever, x_test_gender_city, x_test_cough ), axis=1)

In [64]:
x_train_transformed.shape

(80, 7)

# With Column Transformer

In [67]:
from sklearn.compose import ColumnTransformer

In [83]:
transformer = ColumnTransformer(transformers=[
    ('tnf1', SimpleImputer(), ['fever']),
    ('tnf2', OrdinalEncoder(categories=[['Mild', 'Strong']]), ['cough']),
    ('tnf3', OneHotEncoder(sparse=False, drop="first"), ['gender', 'city']),
], remainder='passthrough')

In [89]:
transformer.fit_transform(x_train)

C:\Users\ADMIN\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[104.        ,   1.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  75.        ],
       [104.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  17.        ],
       [101.        ,   0.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  64.        ],
       [ 98.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  73.        ],
       [101.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  65.        ],
       [100.82191781,   0.        ,   1.        ,   0.        ,
          0.        ,   1.        ,  23.        ],
       [100.82191781,   1.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  79.        ],
       [ 99.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  22.        ],
       [101.        ,   0.        ,   0.        ,   0.        ,
          0.    

In [87]:
transformer.transform(x_test).shape

(20, 7)